**This notebook covered by the following [license](License.ipynb)  This note must not be removed**

# The Postgres Operator by Zalando

see https://github.com/zalando/postgres-operator

In [2]:
pwd

/minikube-host/notebooks/training-kubernetes-security


In [3]:
git clone https://github.com/zalando/postgres-operator.git


Cloning into 'postgres-operator'...
remote: Enumerating objects: 20416, done.        
remote: Counting objects: 100% (312/312), done.        
remote: Compressing objects: 100% (186/186), done.        
remote: Total 20416 (delta 176), reused 209 (delta 119), pack-reused 20104        
Receiving objects: 100% (20416/20416), 7.97 MiB | 21.99 MiB/s, done.
Resolving deltas: 100% (14475/14475), done.


In [6]:
pwd

/minikube-host/notebooks/training-kubernetes-security


In [7]:
cd postgres-operator

In [ ]:
kubectl delete -f manifests/configmap.yaml  # configuration
kubectl delete -f manifests/operator-service-account-rbac.yaml  # identity and permissions
kubectl delete -f manifests/postgres-operator.yaml  # deployment
kubectl delete -f manifests/api-service.yaml  # operator API to be used by UI

In [8]:
helm install postgres-operator ./charts/postgres-operator

manifest_sorter.go:192: info: skipping unknown hook: "crd-install"
manifest_sorter.go:192: info: skipping unknown hook: "crd-install"
manifest_sorter.go:192: info: skipping unknown hook: "crd-install"
NAME: postgres-operator
LAST DEPLOYED: Fri Apr 30 12:13:45 2021
NAMESPACE: default
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
To verify that postgres-operator has started, run:

  kubectl --namespace=default get pods -l "app.kubernetes.io/name=postgres-operator"


In [9]:
kubectl --namespace=default get pods -l "app.kubernetes.io/name=postgres-operator"


NAME                                READY   STATUS    RESTARTS   AGE
postgres-operator-99fb4d44d-78j6j   1/1     Running   0          11s


In [10]:
kubectl get pods -A

NAMESPACE              NAME                                        READY   STATUS    RESTARTS   AGE
default                cloudbomb                                   1/1     Running   0          46h
default                nginx-strict-5cdc48cf55-27r9c               1/1     Running   0          46h
default                nginx-strict-5cdc48cf55-k65vn               1/1     Running   0          46h
default                nginx-strict-5cdc48cf55-rwkt5               1/1     Running   0          46h
default                nginx-with-token-again-66868cff8b-dncj2     1/1     Running   0          46h
default                postgres-operator-99fb4d44d-78j6j           1/1     Running   0          23s
kube-scan              kube-scan-db6ffdfd5-jvjg6                   2/2     Running   0          46h
kube-system            coredns-74ff55c5b-hq2hm                     1/1     Running   1          2d21h
kube-system            etcd-minikube                               1/1     Running   1          2d

In [11]:
helm install postgres-operator-ui ./charts/postgres-operator-ui

NAME: postgres-operator-ui
LAST DEPLOYED: Fri Apr 30 12:14:20 2021
NAMESPACE: default
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
To verify that postgres-operator has started, run:

  kubectl --namespace=default get pods -l "app.kubernetes.io/name=postgres-operator-ui"


In [16]:
kubectl --namespace=default get pods -l "app.kubernetes.io/name=postgres-operator-ui"

NAME                                    READY   STATUS    RESTARTS   AGE
postgres-operator-ui-6bcb7fd597-4hqcw   1/1     Running   0          29s


In [12]:
kubectl port-forward svc/postgres-operator-ui 8081:80

Forwarding from 127.0.0.1:8081 -> 8081


In [ ]:
kubectl get all

In [ ]:
kubectl delete deployment nginx-deployment nginx-other-deployment nginx-strict nginx-hostnetwork-deployment

# In a separate terminal

`kubectl port-forward --address $(hostname -i) svc/postgres-operator-ui 8081:80`

In [14]:
kubectl get secret

NAME                                                      TYPE                                  DATA   AGE
default-token-kvx2n                                       kubernetes.io/service-account-token   3      17h
postgres-operator-token-m2fqq                             kubernetes.io/service-account-token   3      14m
postgres-operator-ui-token-hv7wn                          kubernetes.io/service-account-token   3      14m
postgres-pod-token-68vgp                                  kubernetes.io/service-account-token   3      97s
postgres.acid-mypg.credentials.postgresql.acid.zalan.do   Opaque                                2      97s
sh.helm.release.v1.postgres-operator-ui.v1                helm.sh/release.v1                    1      14m
sh.helm.release.v1.postgres-operator.v1                   helm.sh/release.v1                    1      15m
standby.acid-mypg.credentials.postgresql.acid.zalan.do    Opaque                                2      97s


In [18]:
kubectl get svc 

NAME                   TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)    AGE
acid-mypg              ClusterIP   10.97.82.151     <none>        5432/TCP   101s
acid-mypg-config       ClusterIP   None             <none>        <none>     63s
acid-mypg-repl         ClusterIP   10.99.187.3      <none>        5432/TCP   101s
kubernetes             ClusterIP   10.96.0.1        <none>        443/TCP    2d21h
nginx-strict           ClusterIP   10.111.247.228   <none>        80/TCP     47h
postgres-operator      ClusterIP   10.101.37.55     <none>        8080/TCP   5m1s
postgres-operator-ui   ClusterIP   10.108.70.152    <none>        80/TCP     4m26s


In [19]:
kubectl get service acid-mypg-repl -o json

{
    "apiVersion": "v1",
    "kind": "Service",
    "metadata": {
        "creationTimestamp": "2021-04-30T12:17:05Z",
        "labels": {
            "application": "spilo",
            "cluster-name": "acid-mypg",
            "spilo-role": "replica",
            "team": "acid"
        },
        "name": "acid-mypg-repl",
        "namespace": "default",
        "resourceVersion": "147762",
        "uid": "2e9e9234-325c-4e51-97d8-c60b41b47663"
    },
    "spec": {
        "clusterIP": "10.99.187.3",
        "clusterIPs": [
            "10.99.187.3"
        ],
        "ports": [
            {
                "name": "postgresql",
                "port": 5432,
                "protocol": "TCP",
                "targetPort": 5432
            }
        ],
        "selector": {
            "application": "spilo",
            "cluster-name": "acid-mypg",
            "spilo-role": "replica"
        },
        "sessionAffinity": "None",
        "type": "ClusterIP"
    },
    "status": {
     

In [24]:
export PGPASSWORD=$(kubectl get secret postgres.acid-mypg.credentials.postgresql.acid.zalan.do -o 'jsonpath={.data.password}' | base64 -d)
export PGSSLMODE=require

export PGHOST=$(kubectl get service acid-mypg-repl -o jsonpath='{.status.loadBalancer.ingress[0].ip}' )
echo $PGHOST

echo use
PGCOMMAND=$(echo PGPASSWORD=$PGPASSWORD PGSSLMODE=$PGSSLMODE  /usr/bin/psql  -h $PGHOST -U postgres)
echo $PGCOMMAND
echo to connect


use
PGPASSWORD=yY1WYEOj4Cb7403AeyCKeA0U98yBEFIXU2xRwiVh6Mwc7EOHQMUeyXiCf0K3Q2DP PGSSLMODE=require /usr/bin/psql -h -U postgres
to connect


In [21]:
kubectl get service acid-mypg-repl -o jsonpath='{.status.loadBalancer.ingress[*].ip}'

In [22]:
kubectl get all

NAME                                          READY   STATUS    RESTARTS   AGE
pod/acid-mypg-0                               1/1     Running   0          2m8s
pod/acid-mypg-1                               1/1     Running   0          94s
pod/acid-mypg-2                               1/1     Running   0          90s
pod/cloudbomb                                 1/1     Running   0          46h
pod/nginx-strict-5cdc48cf55-27r9c             1/1     Running   0          46h
pod/nginx-strict-5cdc48cf55-k65vn             1/1     Running   0          46h
pod/nginx-strict-5cdc48cf55-rwkt5             1/1     Running   0          46h
pod/nginx-with-token-again-66868cff8b-dncj2   1/1     Running   0          46h
pod/postgres-operator-99fb4d44d-78j6j         1/1     Running   0          5m29s
pod/postgres-operator-ui-6bcb7fd597-4hqcw     1/1     Running   0          4m54s

NAME                           TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)    AGE
service/acid-mypg              Clust

In [23]:
kubectl exec -t  acid-mypg-0 -- /bin/bash -c "$PGCOMMAND -c 'select datname FROM pg_database'"

psql: error: could not translate host name "-U" to address: Name or service not known
command terminated with exit code 2


: 2

In [ ]:
kubectl get pods


In [ ]:
kubectl logs acid-mypg-1 | grep  --color=always -E '$|leader'

In [ ]:
kubectl delete pod acid-mypg-0